In [1]:
from google.colab import  drive #importamos google drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
%cd ..

/


In [3]:
%cd gdrive/MyDrive/Tesis/cp-for-image-classifier/src/models

/gdrive/MyDrive/Tesis/cp-for-image-classifier/src/models


model.eval() is a kind of switch for some specific layers/parts of the model that behave differently during training and inference (evaluating) time. For example, Dropouts Layers, BatchNorm Layers etc. You need to turn off them during model evaluation, and .eval() will do it for you. In addition, the common practice for evaluating/validation is using torch.no_grad() in pair with model.eval() to turn off gradients computation:

In [1]:
#libraries
import glob as glob
from PIL import Image
import os
import torch
import numpy as np
from torchvision import transforms

In [5]:
from torch.autograd import Variable
import torch.nn.functional as F

In [6]:
from model import build_model,build_model_inference
from paths import *

In [7]:
# constants
IMAGE_SIZE = 580
DEVICE = 'cpu'
# Class names.
class_names = ['agouti', 'collared_peccary', 'common_opossum', 'european_hare', 'ocelot', 'paca', 'red_brocket_deer', 'red_fox', 'red_squirrel', 'roe_deer', 'spiny_rat', 'white-nosed_coati', 'white_tailed_deer', 'wild_boar']


In [ ]:
# Load the trained model.
device = ('cuda' if torch.cuda.is_available() else 'cpu')
model = build_model_inference(modelname='efficientnet_b0', num_classes=14, pretrained=True).to(device)  # 'weights'
model.eval()

In [12]:
#all_image_paths = glob.glob(f"{TEST_PATH}/*")
images_path = glob.glob(TEST_PATH+'/*.jpg')

In [13]:
transform = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        # transforms.RandomApply(transforms=
        #                       [
        #                        #transforms.Normalize([0.3768, 0.3809, 0.3522],[0.1951,0.1968,0.1943])
        #                        
        #                       ], p=0.6)
    ])

In [ ]:
def prediction(img_path,transformer):
  image = Image.open(img_path)
  image_tensor = transformer(image).float()
  image_tensor = image_tensor.unsqueeze_(0)
  if torch.cuda.is_available():
    image_tensor.cuda()
  input = Variable(image_tensor)
  with torch.no_grad():
    output = model(input)
  index = output.data.numpy().argmax()
  return index

In [ ]:
#input_batch shape: (64, in_channels, 224, 224)
outputs = []

for i in images_path:
  outputs.append(prediction(i,transform))
  
result = torch.tensor(outputs)

In [ ]:
result

tensor([4, 8, 8])

In [ ]:
def evaluate(model, testloader):
    model.eval()
    print('Validation')
    counter = 0
    preds_list = []
    target_list = []
    with torch.no_grad():
        for i, data in tqdm(enumerate(testloader), total=len(testloader)):
            counter += 1
            image, labels = data
            image = image.to(device)
            labels = labels.to(device)
            # Forward pass.
            outputs = model(image)
            # Calculate the loss.
            #loss = criterion(outputs, labels)
            # Calculate the accuracy.
            _, preds = torch.max(outputs.data, 1)
            preds_list.append(preds)
            target_list.append(labels)
            #valid_running_correct += (preds == labels).sum().item()

    return preds_list, target_list

In [14]:
from torchvision import transforms, datasets
from paths import *
from tqdm.auto import tqdm
BATCH_SIZE = 32
NUM_WORKERS = 0 
dataset_test = datasets.ImageFolder(
        TEST_PATH, 
        transform=(transform)
    )
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

In [17]:
for i, (x, target) in enumerate(test_loader):
  #print(x.shape[0])
  print(target.shape[0])

32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
32
10


In [ ]:
len(dataset_test)

1450

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
device

'cuda'

In [ ]:
preds_list, target_list = evaluate(model, test_loader)

Validation


  0%|          | 0/46 [00:00<?, ?it/s]

In [ ]:
preds = torch.cat(preds_list)
target = torch.cat(target_list)

In [ ]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 529 kB 33.6 MB/s 


In [ ]:
from torchmetrics.functional import precision_recall
precision, recall = precision_recall(preds, target, average='macro', num_classes=14)
#precision_recall(preds, target, average='micro')

(tensor(0.8647, device='cuda:0'), tensor(0.8505, device='cuda:0'))

In [ ]:
predictoion('/gdrive/MyDrive/Tesis/cp-for-image-classifier/data/processed/test/f509dc22-b73c-45e4-88d3-b0d1b47a5f5b.jpg',transform)
#images_path

4

In [ ]:
def prediction(img_path,transformer):
  image = Image.open(img_path)
  image_tensor = transformer(image).float()
  image_tensor = image_tensor.unsqueeze_(0)
  if torch.cuda.is_available():
    image_tensor.cuda()
  input = Variable(image_tensor)
  with torch.no_grad():
    output = model(input)
  index = output.data.numpy().argmax()

  pred = class_names[index]
  return pred

In [ ]:
pred_dict = {}
for i in images_path:
  pred_dict[i[i.rfind('/')+1:]] = prediction(i,transform)

In [ ]:
pred_dict

{'f509dc22-b73c-45e4-88d3-b0d1b47a5f5b.jpg': 'ocelot',
 'ee951e36-3052-4cd6-b96d-323122c53a58.jpg': 'red_squirrel',
 'd45f6363-9bef-4493-afed-7d89580099e8.jpg': 'red_squirrel'}

In [ ]:
#softmax
# Softmax is also in torch.nn.functional
data = torch.randn(5)
print(data)
print(F.softmax(data, dim=0))
print(F.softmax(data, dim=0).sum())  # Sums to 1 because it is a distribution!
print(F.log_softmax(data, dim=0)) 

tensor([-0.0256, -1.1760, -0.5110,  0.3297,  0.9649])
tensor([0.1653, 0.0523, 0.1017, 0.2358, 0.4450])
tensor(1.)
tensor([-1.8003, -2.9507, -2.2856, -1.4449, -0.8098])


In [ ]:
def prediction_probabilities(img_path,transformer):
  image = Image.open(img_path)
  image_tensor = transformer(image).float()
  image_tensor = image_tensor.unsqueeze_(0)
  if torch.cuda.is_available():
    image_tensor.cuda()
  input = Variable(image_tensor)
  with torch.no_grad():
    output = model(input)
  p_output = F.softmax(output, dim=1)
  print(p_output.sum())
  return p_output

In [ ]:
pred_dict = {}
for i in images_path:
  pred_dict[i[i.rfind('/')+1:]] = prediction_probabilities(i,transform)

tensor(1.)
tensor(1.)
tensor(1.)


In [ ]:
pred_dict

{'f509dc22-b73c-45e4-88d3-b0d1b47a5f5b.jpg': tensor([[1.1543e-08, 3.1053e-07, 1.3759e-07, 7.5890e-08, 9.9999e-01, 2.4810e-06,
          1.7626e-08, 4.8626e-07, 1.4720e-06, 2.5890e-09, 5.2520e-08, 3.7588e-09,
          1.3171e-07, 4.9573e-08]]),
 'ee951e36-3052-4cd6-b96d-323122c53a58.jpg': tensor([[5.7048e-05, 6.0200e-08, 3.3360e-03, 1.2637e-06, 1.3268e-08, 8.0940e-08,
          4.0277e-06, 9.8626e-08, 9.9660e-01, 2.3171e-06, 2.4666e-07, 1.3496e-06,
          1.2910e-08, 2.1102e-09]]),
 'd45f6363-9bef-4493-afed-7d89580099e8.jpg': tensor([[4.6284e-04, 9.4898e-07, 4.2593e-03, 2.6637e-06, 1.9091e-06, 2.3238e-06,
          1.6750e-05, 1.7642e-07, 9.9483e-01, 1.5468e-04, 3.3153e-07, 2.7162e-04,
          4.2676e-08, 4.1161e-07]])}

In [ ]:
out = pred_dict['d45f6363-9bef-4493-afed-7d89580099e8.jpg']
out.shape

torch.Size([1, 14])

In [ ]:
sortedIndex = np.argsort(out).flip(dims=(1,)) 
sortedIndex

tensor([[ 8,  2,  0, 11,  9,  6,  3,  5,  4,  1, 13, 10,  7, 12]])

In [ ]:
[class_names[i] for i in sortedIndex[0]]

['red_squirrel',
 'common_opossum',
 'agouti',
 'white-nosed_coati',
 'roe_deer',
 'red_brocket_deer',
 'european_hare',
 'paca',
 'ocelot',
 'collared_peccary',
 'wild_boar',
 'spiny_rat',
 'red_fox',
 'white_tailed_deer']

In [2]:
!python train.py -e 1 -lr 0.01 -bs 8

python3: can't open file 'train.py': [Errno 2] No such file or directory


In [8]:
def get_model_name(model_properties: dict):
  current_model_name = "model"
  for key in model_properties.keys():

    current_model_name += "-"+ key + "_" + str(model_properties[key])
  return current_model_name


In [9]:
model_properties = {
    "batch_size":32,
    "optimizer":"adam",
    "lr":0.001
}
myFun(model_properties)

'model-batch_size_32-optimizer_adam-lr_0.001'